In [1]:
%load_ext autoreload
%autoreload 1

import glob
import numpy as np
import pandas as pd
from collections import OrderedDict

import matplotlib.pyplot as plt
import scipy
%matplotlib widget

processed_directory = './data/processed_mats/*.mat'
# processed_directory = '/Volumes/AnxietyBU/callbacks/processed/*.mat'

acceptable_call_labels = ['Call', 'Stimulus']  # any stimulus_trials containing call types NOT in this list are excluded (this includes unlabeled, which are stored as 'USV'!!)

files = [f for f in glob.glob(processed_directory)]

# or only specific files. Note: may mess up histograms, which may require data from >1 file
# files = [
#     './data/processed_mats/or60rd49-d1-20240425115050-Block1-PROCESSED.mat',
#     './data/processed_mats/or60rd49-d2-20240426114923-Block1-PROCESSED.mat'
# ]
files


['./data/processed_mats/pk81rd39-d2-20240403122742-Block6-PROCESSED.mat',
 './data/processed_mats/or54rd45-d1-20240423130940-Block0 2024-07-16  7_34 AM-PROCESSED.mat',
 './data/processed_mats/gr18bu69-d2-20240419122717-Block3-PROCESSED.mat',
 './data/processed_mats/pk63rd15-d1-202431212348-Block7 2024-07-16  9_04 AM-PROCESSED.mat',
 './data/processed_mats/gr18bu69-d2-20240419123353-Block8-PROCESSED.mat',
 './data/processed_mats/gr18bu69-d2-20240419122320-Block0-PROCESSED.mat',
 './data/processed_mats/or14pu27-d1-2024329115821-Block1-PROCESSED.mat',
 './data/processed_mats/or14pu27-d2-2024330115030-Block0-PROCESSED.mat',
 './data/processed_mats/or60rd49-d1-20240425114931-Block0 2024-07-17 10_20 AM-PROCESSED.mat',
 './data/processed_mats/pk81rd39-d2-20240403122107-Block1-PROCESSED.mat',
 './data/processed_mats/pk81rd39-d3-20240404160754-Block6 2024-07-29  1_12 PM-PROCESSED.mat',
 './data/processed_mats/or14pu27-d2-2024330115825-Block6-PROCESSED.mat',
 './data/processed_mats/or91rd13-d1-2

In [2]:
%aimport utils.deepsqueak
from utils.deepsqueak import call_mat_stim_trial_loader, multi_index_from_dict

df = pd.DataFrame()

call_types_all = pd.DataFrame()
rejected_trials_all = pd.DataFrame()

for file in files:
    calls_df, stim_trials, rejected_trials, file_info, call_types = call_mat_stim_trial_loader(file, acceptable_call_labels=['Call', 'Stimulus'], verbose=False)

    # TODO: make this a nicely editable parameter
    multi_index_info = OrderedDict()
    multi_index_info['birdname'] = file_info['birdname']
    multi_index_info['day'] = int(file_info['d'])
    multi_index_info['block'] = int(file_info['block'])

    # create multiindex: birdname, stim_trial_index, call_index
    stim_trials = multi_index_from_dict(
        stim_trials, 
        multi_index_info, 
        keep_current_index=True,
    )
    df = pd.concat((df, stim_trials), axis='rows')
    
    rejected_trials = multi_index_from_dict(
        rejected_trials, 
        multi_index_info, 
        keep_current_index=True
    )
    rejected_trials_all = pd.concat((rejected_trials_all, rejected_trials), axis='rows')

    call_types = multi_index_from_dict(
        call_types, 
        multi_index_info, 
        keep_current_index=True
    )
    call_types_all = pd.concat((call_types_all, call_types), axis='rows')

print('Rejected trials:')
rejected_trials_all

Rejected trials:


trial_start_s  trial_end_s  stim_duration_s  \
birdname day block calls_index                                                
pk81rd39 2   6     7                 4.544741     6.382694         0.122577   
gr18bu69 2   3     1                 5.023158     6.784143         0.124337   
or14pu27 1   1     1                 3.809568     4.836454         0.155488   
         2   0     0                 4.457682     7.431893         0.148597   
                   23               18.128280    19.512110         0.151198   
...                                       ...          ...              ...   
pk81rd39 1   1     24               17.194818    19.341300         0.146826   
                   25               19.341300    22.966062         0.145029   
                   26               22.966062    25.096136         0.142694   
gr18bu69 2   5     1                 4.659141     7.238099         0.109070   
or54rd45 2   3     1                 4.209684     6.985771         0.117460   

                                 calls_in_range  
birdname day block calls_index                   
pk81rd39 2   6     7                     [6, 8]  
gr18bu69 2   3     1                        [0]  
or14pu27 1   1     1            [0, 2, 3, 4, 5]  
         2   0     0               [1, 2, 3, 4]  
                   23              [24, 25, 26]  
...                                         ...  
pk81rd39 1   1     24                      [23]  
                   25                      [23]  
                   26                      [23]  
gr18bu69 2   5     1                        [0]  
or54rd45 2   3     1                        [0]  

[179 rows x 4 columns]

In [3]:
print(
    "Call types in rejected trials."
    + "\nLabel `USV` means an accepted call was not given a label."
    + "\nGo back to DeepSqueak & fix ths."
)

rejected_trial_call_types = call_types_all.loc[rejected_trials_all.index]
rejected_trial_call_types
# TODO: add stim index to rej trial type df (is this the first stim?)

# # see only blocks with a specific call type
#
# label = 'USV'
# label = 'Noise'
# call_types_all.loc[~np.isnan(call_types_all.loc[:, label])]

Call types in rejected trials.
Label `USV` means an accepted call was not given a label.
Go back to DeepSqueak & fix ths.


type                            Loom_wing_flap  Call  Song
birdname day block calls_index                            
pk81rd39 2   6     7                       1.0   1.0   NaN
gr18bu69 2   3     1                       1.0   NaN   NaN
or14pu27 1   1     1                       NaN   4.0   1.0
         2   0     0                       1.0   3.0   NaN
                   23                      NaN   2.0   1.0
...                                        ...   ...   ...
pk81rd39 1   1     24                      NaN   NaN   1.0
                   25                      NaN   NaN   1.0
                   26                      NaN   NaN   1.0
gr18bu69 2   5     1                       1.0   NaN   NaN
or54rd45 2   3     1                       1.0   NaN   NaN

[179 rows x 3 columns]

In [4]:
df.sort_index(inplace=True)
df

trial_start_s  trial_end_s  stim_duration_s  \
birdname day block stims_index                                                
gr18bu69 1   0     0                 4.539624     6.674403         0.155215   
                   1                 6.674403     9.191932         0.151767   
                   2                 9.191932    11.604473         0.139421   
                   3                11.604473    14.214170         0.150624   
                   4                14.214170    15.231342         0.135456   
...                                       ...          ...              ...   
pk81rd39 3   9     9                24.033292    27.098310         0.143663   
                   10               27.098310    29.239676         0.147698   
                   11               29.239676    31.098234         0.142542   
                   12               31.098234    32.862232         0.155216   
                   13               32.862232    34.086893         0.151020   

                                          calls_in_range  \
birdname day block stims_index                             
gr18bu69 1   0     0                              [1, 2]   
                   1              [4, 5, 6, 7, 8, 9, 10]   
                   2                [12, 13, 14, 15, 16]   
                   3            [18, 19, 20, 21, 22, 23]   
                   4                    [23, 25, 26, 27]   
...                                                  ...   
pk81rd39 3   9     9                            [27, 28]   
                   10                           [30, 31]   
                   11                           [33, 34]   
                   12                           [36, 37]   
                   13                               [39]   

                                                          call_times_stim_aligned  \
birdname day block stims_index                                                      
gr18bu69 1   0     0            [[0.557610034942627, 0.6310633964061951], [1.2...   
                   1            [[0.193976288810771, 0.31964473800863225], [0....   
                   2            [[0.15544715157167133, 0.3274423145355385], [0...   
                   3            [[0.2118062858275529, 0.3161215140432798], [0....   
                   4            [[-0.16002429700365184, 0.010610820421206313],...   
...                                                                           ...   
pk81rd39 3   9     9            [[0.14057834468293606, 0.19134513474110904], [...   
                   10           [[0.15032897262813094, 0.19537873690261875], [...   
                   11           [[0.14460774976711477, 0.2231829200133184], [1...   
                   12           [[0.19690878295099168, 0.2754839531971953], [1...   
                   13                  [[0.1614188778269252, 0.2483753995660578]]   

                                n_calls  latency_s  \
birdname day block stims_index                       
gr18bu69 1   0     0                  2   0.557610   
                   1                  7   0.193976   
                   2                  5   0.155447   
                   3                  6   0.211806   
                   4                  3   0.265394   
...                                 ...        ...   
pk81rd39 3   9     9                  2   0.140578   
                   10                 2   0.150329   
                   11                 2   0.144608   
                   12                 2   0.196909   
                   13                 1   0.161419   

                                                                     wav_filename  \
birdname day block stims_index                                                      
gr18bu69 1   0     0            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   1            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   2            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-b

In [5]:
all_birds = list(set(df.index.get_level_values(0)))
all_birds

['gr3bu36',
 'or54rd45',
 'or91rd13',
 'gr18bu69',
 'or14pu27',
 'gr44bu34',
 'or60rd49',
 'pk63rd15',
 'pk81rd39']

In [6]:
# eliminate all block 0s - account for first loom bug

# raise Exception('Make sure you want to do this! You will need to reload the data afterward if you want block 0 back.')

# blocks = df.index.get_level_values(2)
# df = df[blocks != 0]

df

trial_start_s  trial_end_s  stim_duration_s  \
birdname day block stims_index                                                
gr18bu69 1   0     0                 4.539624     6.674403         0.155215   
                   1                 6.674403     9.191932         0.151767   
                   2                 9.191932    11.604473         0.139421   
                   3                11.604473    14.214170         0.150624   
                   4                14.214170    15.231342         0.135456   
...                                       ...          ...              ...   
pk81rd39 3   9     9                24.033292    27.098310         0.143663   
                   10               27.098310    29.239676         0.147698   
                   11               29.239676    31.098234         0.142542   
                   12               31.098234    32.862232         0.155216   
                   13               32.862232    34.086893         0.151020   

                                          calls_in_range  \
birdname day block stims_index                             
gr18bu69 1   0     0                              [1, 2]   
                   1              [4, 5, 6, 7, 8, 9, 10]   
                   2                [12, 13, 14, 15, 16]   
                   3            [18, 19, 20, 21, 22, 23]   
                   4                    [23, 25, 26, 27]   
...                                                  ...   
pk81rd39 3   9     9                            [27, 28]   
                   10                           [30, 31]   
                   11                           [33, 34]   
                   12                           [36, 37]   
                   13                               [39]   

                                                          call_times_stim_aligned  \
birdname day block stims_index                                                      
gr18bu69 1   0     0            [[0.557610034942627, 0.6310633964061951], [1.2...   
                   1            [[0.193976288810771, 0.31964473800863225], [0....   
                   2            [[0.15544715157167133, 0.3274423145355385], [0...   
                   3            [[0.2118062858275529, 0.3161215140432798], [0....   
                   4            [[-0.16002429700365184, 0.010610820421206313],...   
...                                                                           ...   
pk81rd39 3   9     9            [[0.14057834468293606, 0.19134513474110904], [...   
                   10           [[0.15032897262813094, 0.19537873690261875], [...   
                   11           [[0.14460774976711477, 0.2231829200133184], [1...   
                   12           [[0.19690878295099168, 0.2754839531971953], [1...   
                   13                  [[0.1614188778269252, 0.2483753995660578]]   

                                n_calls  latency_s  \
birdname day block stims_index                       
gr18bu69 1   0     0                  2   0.557610   
                   1                  7   0.193976   
                   2                  5   0.155447   
                   3                  6   0.211806   
                   4                  3   0.265394   
...                                 ...        ...   
pk81rd39 3   9     9                  2   0.140578   
                   10                 2   0.150329   
                   11                 2   0.144608   
                   12                 2   0.196909   
                   13                 1   0.161419   

                                                                     wav_filename  \
birdname day block stims_index                                                      
gr18bu69 1   0     0            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   1            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   2            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-b

In [8]:
save_data_path = None
save_data_path = "./data/data.pickle"

if save_data_path is not None:
    import pickle

    to_save = dict(
        all_birds=all_birds,
        df=df,
        rejected_trials_all=rejected_trials_all,
        call_types_all=call_types_all,
    )

    with open(save_data_path, "wb") as f:
        pickle.dump(to_save, file=f)

print(f"Saved to: {save_data_path}")

Saved to: ./data/data.pickle
